# Tutorial 02: Working with Language Models in LangChain

In this tutorial, we'll explore how to work with language models in LangChain, focusing on the Groq LLM. We'll cover connecting to the model, creating prompt templates, building chains, and handling responses.

## 1. Connecting to Language Models

First, let's set up our environment and connect to the Groq LLM:

In [2]:
import os
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv

load_dotenv()

# Initialize the Groq LLM
llm = ChatGroq(
        model_name="qwen-2.5-32b",
        temperature=0.1,
        model_kwargs={"top_p": 0.2, "seed": 1337}
    )

# Test the connection
response = llm.invoke("Hello, world!")
print(response)

content='Hello! How can I assist you today?' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 33, 'total_tokens': 43, 'completion_time': 0.05, 'prompt_time': 0.003967551, 'queue_time': 0.047193639, 'total_time': 0.053967551}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_92412bc7e4', 'finish_reason': 'stop', 'logprobs': None} id='run-ed95a062-2ed6-4445-a37a-6eb2fd2937a6-0' usage_metadata={'input_tokens': 33, 'output_tokens': 10, 'total_tokens': 43}


## 2. Creating Prompt Templates

Prompt templates allow us to create reusable prompts with input variables:

In [3]:
# Define a simple prompt template
template = """Answer the following question:
You answers in the {language} language.
Question: {question}
Answer: Let's approach this step-by-step:"""


prompt = PromptTemplate(template=template, input_variables=["question","language"])

# Use the prompt template
question = "What is the capital of France?"
formatted_prompt = prompt.format(question=question,language="English")
print(formatted_prompt)

Answer the following question:
You answers in the English language.
Question: What is the capital of France?
Answer: Let's approach this step-by-step:


## 3. Building Simple Prompt Chains

Now, let's create a chain that combines our prompt template with the language model:

In [4]:
# Create a chain
chain = prompt | llm

# Run the chain
result = chain.invoke({"question":"What is the speed of light?","language":"French"})
print(result.content)

La vitesse de la lumière dans le vide est d'environ 299 792 kilomètres par seconde. Cette valeur est souvent arrondie à 300 000 kilomètres par seconde pour des calculs plus simples. Cette constante est fondamentale en physique et est notée par la lettre "c".


## 4. Handling Model Responses

Let's explore different ways to handle and process model responses:

In [5]:
# Get the raw response
raw_response = llm.invoke("List three prime numbers.")
print("Raw response:", raw_response)

# Using the chain with a dictionary input
chain_response = chain.invoke({"question": "tree Names","language": "French"})
print("\nChain response:", chain_response.content)

Raw response: content='Here are three prime numbers:\n\n1. 2\n2. 3\n3. 5\n\nThese numbers are prime because they are only divisible by 1 and themselves.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 34, 'total_tokens': 71, 'completion_time': 0.185, 'prompt_time': 0.004139689, 'queue_time': 0.04928391, 'total_time': 0.189139689}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_92412bc7e4', 'finish_reason': 'stop', 'logprobs': None} id='run-a9390958-2343-46e9-9280-93e5647e8f92-0' usage_metadata={'input_tokens': 34, 'output_tokens': 37, 'total_tokens': 71}

Chain response: Bien sûr, je vais vous donner quelques noms d'arbres en français :

1. Chêne
2. Hêtre
3. Bouleau
4. Pin
5. Sapin
6. Orme
7. Peuplier
8. Érable
9. Châtaignier
10. Noisetier

Ces arbres sont couramment trouvés en France et dans d'autres pays européens. Chacun a ses propres caractéristiques et utilisations.


In [6]:
# Parsing structured output
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

list_prompt = PromptTemplate(
    template="List 100 {item}. {format_instructions} write only the colors, nothing else",
    input_variables=["item"],
    partial_variables={"format_instructions": format_instructions}
)

chain = list_prompt | llm |output_parser
result = chain.invoke({"item":"colors"})
print(type(result))
print("\nParsed list:", result)

<class 'list'>

Parsed list: ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet', 'pink', 'purple', 'brown', 'black', 'white', 'gray', 'silver', 'gold', 'beige', 'maroon', 'olive', 'cyan', 'magenta', 'teal', 'navy', 'azure', 'lavender', 'rose', 'scarlet', 'crimson', 'mauve', 'taupe', 'chartreuse', 'periwinkle', 'coral', 'peach', 'tangerine', 'lemon', 'emerald', 'forest', 'sky', 'cerulean', 'amethyst', 'rosewood', 'ivory', 'ecru', 'flax', 'taupe', 'umber', 'brick', 'claret', 'crimson', 'ochre', 'vermilion', 'azure', 'cerulean', 'chartreuse', 'crimson', 'cyan', 'emerald', 'forest', 'indigo', 'ivory', 'azure', 'beige', 'black', 'blue', 'blue-gray', 'blue-green', 'blue-violet', 'brown', 'brown-orange', 'bronze', 'brun', 'burgundy', 'byzantium', 'cerise', 'cerulean', 'chestnut', 'chrome-yellow', 'cobalt', 'coffee', 'cool-gray', 'copper', 'copper-red', 'coral', 'coral-pink', 'coral-red', 'cream', 'cyan', 'deep-pink', 'deep-purple', 'deep-salmon', 'deep-sky-blue', 'denim', 'desert

## 5. Best Practices for Prompt Engineering

Here are some best practices for effective prompt engineering:

In [7]:
# 1. Be specific and provide context
specific_prompt = PromptTemplate(
    template="You are an expert in {field}. Explain {concept} in simple terms for a beginner.",
    input_variables=["field", "concept"]
)

# 2. Use examples (few-shot learning)
few_shot_prompt = PromptTemplate(
    template="""Classify the sentiment of the following text as positive, negative, or neutral.

Example 1:
Text: I love this product!
Sentiment: Positive

Example 2:
Text: This is the worst experience ever.
Sentiment: Negative

Example 3:
Text: The weather is cloudy today.
Sentiment: Neutral

Now, classify the following text:
Text: {text}
Sentiment:""",
    input_variables=["text"]
)

# 3. Break complex tasks into steps
step_prompt = PromptTemplate(
    template="""To solve the problem '{problem}', follow these steps:
1. Identify the key information
2. Determine the appropriate formula or method
3. Apply the formula or method step-by-step
4. Check your answer

Now, solve the problem:""",
    input_variables=["problem"]
)

# Test the prompts
chains = {
    "Specific": specific_prompt | llm,
    "Few-shot": few_shot_prompt | llm,
    "Step-by-step": step_prompt | llm 
    }

for name, chain in chains.items():
    print(f"\n---------------------------------------------------\n{name} Prompt Result:")
    if name == "Specific":
        print(chain.invoke({"field":"physics", "concept":"quantum entanglement"}).content)
    elif name == "Few-shot":
        print(chain.invoke({"text":"This movie was okay, I guess."}).content)
    else:
        print(chain.invoke({"problem":"Calculate the area of a circle with radius 5 cm"}).content)


---------------------------------------------------
Specific Prompt Result:
Quantum entanglement is a fascinating and somewhat mysterious phenomenon in quantum physics. To understand it, let's start with a simple analogy and then move to the actual concept.

Imagine you have two dice, and you decide to play a game. You roll the dice, but there's a twist: no matter how far apart you take these dice, if one shows a 6, the other will always show a 1, and vice versa. This is a bit like entanglement, but in the quantum world, it's even more surprising because the dice (or particles) don't have to be rolled (or measured) to know what the other will show. They are connected in such a way that the state of one instantly influences the state of the other, no matter how far apart they are.

In quantum physics, particles like electrons or photons can become entangled. When two particles are entangled, the quantum state of one particle is directly related to the quantum state of the other, no mat

## Conclusion

In this tutorial, we've explored various aspects of working with language models in LangChain, including connecting to models, creating prompt templates, building chains, handling responses, and implementing best practices for prompt engineering. These skills will serve as a foundation for building more complex applications with LangChain in future tutorials.